In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
WITH usa AS (
SELECT * FROM invoice i
WHERE billing_country = "USA"
),
usa_track AS (
SELECT * FROM usa u
INNER JOIN invoice_line il ON il.invoice_id = u.invoice_id)

SELECT 
    ar.name artist,
    al.title album,
    g.name genre,
    SUM(il.quantity) num_sold,
    ROUND(SUM(u.total),2) total_sales
FROM usa u
INNER JOIN invoice_line il ON il.invoice_id = u.invoice_id
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar ON ar.artist_id = al.artist_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 2
ORDER BY 5 DESC
LIMIT 3;

Done.


artist,album,genre,num_sold,total_sales
Marvin Gaye,Seek And Shall Find: More Of The Best (1963-1981),R&B/Soul,26,387.09
Nirvana,From The Muddy Banks Of The Wishkah [live],Rock,28,379.17
Jimi Hendrix,Are You Experienced?,Rock,28,370.26


In [3]:
%%sql
SELECT 
    e.first_name || " " || e.last_name agent_name,
    e.title,
    e.hire_date,
    COUNT(DISTINCT c.customer_id) num_customers,
    ROUND(SUM(i.total),2) total_dollars,
    SUM(il.quantity) qty_sales,
    ROUND(ROUND(SUM(i.total),2)/ COUNT(DISTINCT c.customer_id),2) sales_per_customer
FROM customer c
INNER JOIN employee e ON e.employee_id = c.support_rep_id
INNER JOIN invoice i ON i.customer_id = c.customer_id
INNER JOIN invoice_line il on il.invoice_id = i.invoice_id
GROUP BY 1;

Done.


agent_name,title,hire_date,num_customers,total_dollars,qty_sales,sales_per_customer
Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,21,18246.69,1749,868.89
Margaret Park,Sales Support Agent,2017-05-03 00:00:00,20,15923.16,1600,796.16
Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,18,13333.32,1408,740.74


In [4]:
%%sql
WITH country_or_others AS 
 (SELECT CASE 
          WHEN (
              SELECT COUNT(*)
              FROM customer
              WHERE country= c.country
              GROUP BY country) = 1 THEN 'Other'
    ELSE c.country 
    END AS country,
    i.*
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    )

SELECT co.country, 
    COUNT(DISTINCT customer_id) 'No of Customers', 
    ROUND(CAST(SUM(total) AS FLOAT), 2) 'Total Sales',
    ROUND(CAST(SUM(total) AS FLOAT) / COUNT(DISTINCT customer_id), 2) 'Average Sales Value per Customer',
    ROUND(CAST(AVG(total) AS FLOAT), 2) 'Average Order'
FROM (
        SELECT cc.*,
            CASE
                WHEN country = 'Other' THEN 1
                ELSE 0
            END AS sort
        FROM country_or_others cc
    ) co

GROUP BY country
ORDER BY sort, 3 DESC;

Done.


country,No of Customers,Total Sales,Average Sales Value per Customer,Average Order
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.57,8.72
Other,15,1094.94,73.0,7.45


In [13]:
%%sql
WITH tracks_per_album AS
(
    SELECT a.album_id,
    COUNT(t.track_id) track_count
    FROM track t
    INNER JOIN album a ON a.album_id = t.album_id
    GROUP BY a.album_id
),
count_of_tracks AS
(
    SELECT il.invoice_id,
    t.album_id,
    COUNT(il.track_id) track_count
    FROM track t 
    INNER JOIN invoice_line il ON il.track_id = t.track_id
    GROUP BY invoice_id, album_id
)

SELECT purchase_type,
       COUNT(DISTINCT il.invoice_id) invoice_count,
       ROUND(CAST(COUNT(DISTINCT pt.invoice_id) AS FLOAT) / (
                                                       SELECT COUNT(*) 
                                                       FROM invoice
                                                             ) * 100, 2) || "%"percentage
FROM 
(
    SELECT il.invoice_id,
       MIN(il.track_id),
    CASE WHEN
       ct.track_count = tpa.track_count THEN 'Album'
    ELSE 'Individual Tracks'
    END AS purchase_type
    FROM invoice_line il
    INNER JOIN count_of_tracks ct ON il.invoice_id = ct.invoice_id
    INNER JOIN tracks_per_album tpa ON ct.album_id = tpa.album_id
    GROUP BY il.invoice_id
) AS pt
       INNER JOIN invoice_line il ON il.invoice_id = pt.invoice_id
    GROUP BY purchase_type;
       

Done.


purchase_type,invoice_count,percentage
Album,117,19.06%
Individual Tracks,497,80.94%
